In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()

spark

In [37]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSVtoDF").getOrCreate()
from pyspark.sql.functions import col, sum

In [4]:
rural = spark.read.option("header", "true").option("inferSchema", "true").csv("C:\\Users\\pankapra\\Desktop\\Sprint\\FINAL_POPULATION_RURAL.csv").alias("r")
urban = spark.read.option("header","true").option("inferSchema", "true").csv("C:\\Users\\pankapra\\Desktop\\Sprint\\FINAL_POPULATION_URBAN.csv").alias("u")

In [5]:
df = rural.join(urban, rural["COUNTRY ID"] == urban["COUNTRY ID"]).drop(urban["COUNTRY"]).drop(urban["COUNTRY ID"]).drop(urban["CONTINENT"]).drop(urban["DEVELOPMENT_CATEGORY"]).drop(urban["GOVERNANCE"])

In [8]:
df.show()

+--------------------+----------+---------+--------------+--------------------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|             COUNTRY|COUNTRY ID|CONTINENT|   DEVELOPMENT|          GOVERNANCE| TYPE|D_1950|D_1955|D_1960|D_1965|D_1970|D_1975|D_1980|D_1985|D_1990|D_1995|D_2000|D_2005|D_2010|D_2015|D_2020|D_2025|D_2030|D_2035|D_2040|D_2045|D_2050| TYPE|D_1950|D_1955|D_1960|D_1965|D_1970|D_1975|D_1980|D_1985|D_1990|D_1995|D_2000|D_2005|D_2010|D_2015|D_2020|D_2025|D_2030|D_2035|D_2040|D_2045|D_2050|
+--------------------+----------+---------+--------------+--------------------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---

In [9]:
maindf = df.fillna(0)

In [11]:
maindf.show()

+--------------------+----------+---------+--------------+--------------------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|             COUNTRY|COUNTRY ID|CONTINENT|   DEVELOPMENT|          GOVERNANCE| TYPE|D_1950|D_1955|D_1960|D_1965|D_1970|D_1975|D_1980|D_1985|D_1990|D_1995|D_2000|D_2005|D_2010|D_2015|D_2020|D_2025|D_2030|D_2035|D_2040|D_2045|D_2050| TYPE|D_1950|D_1955|D_1960|D_1965|D_1970|D_1975|D_1980|D_1985|D_1990|D_1995|D_2000|D_2005|D_2010|D_2015|D_2020|D_2025|D_2030|D_2035|D_2040|D_2045|D_2050|
+--------------------+----------+---------+--------------+--------------------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---

In [41]:
pop = maindf.cache()

In [42]:
pop.rdd.getNumPartitions()

1

In [17]:
# Ques 4: Urban population of Countries which are Less Developed.

q4 = urban.select(col("COUNTRY"), col("D_2025").alias("Urban_Population")).filter(col("DEVELOPMENT_CATEGORY") == "LESS DEVELOPED")

q4.show()

+--------------------+----------------+
|             COUNTRY|Urban_Population|
+--------------------+----------------+
|             Burundi|            1674|
|             Comoros|             320|
|            Djibouti|             919|
|             Eritrea|            2174|
|            Ethiopia|           23758|
|               Kenya|           17969|
|          Madagascar|           12972|
|              Malawi|            4577|
|           Mauritius|             603|
|             Mayotte|             159|
|          Mozambique|           11685|
|             R�union|             935|
|              Rwanda|            3854|
|          Seychelles|              54|
|             Somalia|            6375|
|         South Sudan|            3186|
|              Uganda|           11526|
|United Republic o...|           24074|
|              Zambia|            9602|
|            Zimbabwe|            7668|
+--------------------+----------------+
only showing top 20 rows



In [18]:
# Ques 5: Which countries are projected to experience a decline in rural population between 1970 and 2025?


decrural = rural.select("COUNTRY", "D_1970", "D_2025").filter(col("D_2025") < col("D_1970"))

decrural.show()


+--------------------+------+------+
|             COUNTRY|D_1970|D_2025|
+--------------------+------+------+
|             R�union|   270|    38|
|               Gabon|   360|   214|
|             Algeria|  8316|  7697|
|          Cape Verde|   221|   163|
|        Saint Helena|     3|     2|
|               China|672878|483453|
|               Japan| 29167|  4581|
|   Republic of Korea| 18644|  6870|
|Other non-specifi...|  9546|  5256|
|            Malaysia|  7260|  7164|
|              Cyprus|   364|   333|
|             Georgia|  2448|  1733|
|              Kuwait|   108|    58|
|             Lebanon|   999|   518|
|               Qatar|    13|     8|
|              Turkey| 21905| 15757|
|             Belarus|  5061|  1827|
|            Bulgaria|  4050|  1379|
|      Czech Republic|  3485|  2797|
|             Hungary|  4115|  2415|
+--------------------+------+------+
only showing top 20 rows



In [44]:
# Ques 10: Continent Wise Lowest Total Population.


df_total = df.withColumn("Total_2025", col("r.D_2025").cast("int") + col("u.D_2025").cast("int"))

cont_totals = df_total.groupBy("CONTINENT").agg(sum("Total_2025").alias("Total_2025"))

lowest = cont_totals.orderBy("Total_2025").limit(1)

lowest.show()


+---------+----------+
|CONTINENT|Total_2025|
+---------+----------+
|  Oceania|     44626|
+---------+----------+

